In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import DictVectorizer

In [2]:
yColumn = "Survived"
dfTrain = pd.read_csv("./train.csv")
dfTest = pd.read_csv("./test.csv")
# dfTrain = pd.read_csv("./train.csv",na_values=[' ',''])
# dfTest = pd.read_csv("./test.csv",na_values=[' ',''])

y = dfTrain[yColumn]
dfTrain.drop(yColumn, axis=1, inplace=True)
X_full = dfTrain.append(dfTest, ignore_index=True)
#X_full.reindex(["PassengerId"],axis="columns")
# X_split_train = X_full.iloc[0:891]
# X_split_test = X_full.iloc[891:1309]

In [3]:
def adjustDataframe(dataFrame):
##  Get name prefix
    namePrefix = dataFrame["Name"].str.split(', ', 1).str[1]
    namePrefix = namePrefix.str.split('. ',1).str[0]
    dataFrame["NamePrefix"] = namePrefix
##  Split ticket into numeric and prefix parts
    spacedTicket = ' ' + dataFrame['Ticket'].astype(str)
    ticketSplit = spacedTicket.str.rsplit(' ',1,expand=True)
    dataFrame["TicketPrefix"] = ticketSplit[0]
    dataFrame["TicketPrefix"] = dataFrame["TicketPrefix"].str.replace(".","")
    dataFrame["TicketPrefix"] = dataFrame["TicketPrefix"].str.replace(" ","")
    dataFrame["TicketNumber"] = ticketSplit[1]  
## There are multiple cabins per some of passengers - how to proceed?
# For now I will only use deck name and amount of cabins. For later I should check how could I use multiple entries
    dataFrame["CabinDeck"] = dataFrame["Cabin"].str[:1]
    dataFrame["CabinsAmount"] = dataFrame["Cabin"].str.split(" ",-1).str.len()

    del namePrefix, spacedTicket, ticketSplit
    dataFrame.drop(["Name", "Ticket", "Cabin"], axis=1, inplace=True)
    return dataFrame

In [4]:
## Fare and Pclass have highest correlations. Low correlations: Age, Parch.  SibSp and PassengerId correlation < 0.05
#XTrain.corrwith(y)
X_full = adjustDataframe(X_full)
# X_full = X_full.replace(r'^\s*$', np.nan, regex=True)
X_full = X_full.replace(r'^\s*$', np.nan, regex=True)

#### Remove Nan values in all relevant columns

In [5]:
numericColumns = ["Pclass", "Age", "SibSp", "Parch", "Fare", "CabinsAmount"]
categoricalColumns = ["Sex","NamePrefix", "TicketPrefix", "CabinDeck", "Embarked"]

mean_Imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_numeric = mean_Imputer.fit_transform(X_full[numericColumns])
categorical_data = X_full[categoricalColumns].fillna("N/A").astype(str)
encoder = DictVectorizer(sparse = False)
X_categorical = encoder.fit_transform(categorical_data.T.to_dict().values())